# Setup

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
from IPython.display import clear_output

!pip3 install pyprind
!pip3 install torchmetrics
clear_output()

ResNet - 50  and Vgg19 Implementation

In [1]:
# Imports

import torch
import os
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets, transforms
import matplotlib.pyplot as plt

A template has been provided to help you. 

* You may or may not edit the `None` fields. There is no need to change anything else.
* You might or might not have to add additional lines other than what is provided. 

In [ ]:
 ''' Need to change transformations and Dense layer a little bit based on whether we using my model or vgg19
 * my model works in image size 32 x 32 but vgg19 works with 224 x 224 
 *  my model reduces image size by 8 x 8 but vgg19 changes it by 7 x 7 
 *  So in dense layer we need to change the input vector acc to that 
 * Need not to specify , we need to change conv layer in forward Fn based on which model we use '''

In [ ]:
# Downloading and Preparing the Dataset

!gdown --id 1oYnD7Izl3LVVzjEMyLxLklX30TKWHgGG
!unzip /content/cifar-10.zip
!rm -rf /content/cifar-10.zip
!mv /content/cifar-10/sample_submission.csv /content/cifar-10/test_labels.csv

clear_output()

In [ ]:
# Imports

import torch
import torchvision
import torchmetrics

from PIL import Image

import pandas 
import numpy
from sklearn import preprocessing
import matplotlib

import os
import pyprind
import torchvision.datasets as datasets


PATH = "/content/" # Need to Add your google drive

In [ ]:

class CreateDataset(torch.utils.data.Dataset):
    def __init__(self, root_dir, mode='train'):
        self.root_dir = root_dir
        self.mode = mode

        self.entry = pandas.read_csv(os.path.join(self.root_dir, f'{self.mode}_labels.csv'))
        self.encoder = self._process_()
        self.entry['label'] = self.encoder.transform(self.entry['label'])
        self.transform = torchvision.transforms.Compose(
            [
                ''' Added some Transforms '''
                torchvision.transforms.RandomInvert(),
                torchvision.transforms.RandomHorizontalFlip(),
                torchvision.transforms.RandomRotation(30),
                #torchvision.transforms.Grayscale(),
                torchvision.transforms.Resize((224,224)), 
                torchvision.transforms.ToTensor()
            
            ])
        
        self.trans = torchvision.transforms.Compose(
            [
                torchvision.transforms.Resize((224,224)), 
                torchvision.transforms.ToTensor()
            ])

    def _process_(self):
        data = pandas.read_csv(os.path.join(self.root_dir, 'train_labels.csv'))
        encoder = preprocessing.LabelEncoder()
        encoder.fit(data['label'])
        return encoder

    def __getitem__(self, index):
        index = index if index !=0 else 1
        data = self.entry.iloc[index]
        img_path = '/content/cifar-10/'+ self.mode + '/' + str(index) + '.png'
        image = plt.imread(img_path) # Read Image?    ( Bro here index of images are random , so what to do ?)  - yes done
        image = Image.fromarray(image, "RGB")
        image = self.trans(image)
        label = data[1]                 # Read Label? - yes done 
        return image, label
      


    def __len__(self):
        return len(self.entry)

In [ ]:
k = CreateDataset("/content/cifar-10")
trainloader22 = torch.utils.data.DataLoader(k, batch_size=5000, shuffle=True, drop_last=True)

In [ ]:
# getting some info about image
sizes = []
for image in os.listdir("/content/cifar-10/train"):
  image = plt.imread("/content/cifar-10/train/"+image)
  sizes.append(image.shape)
pandas.Series(sizes).value_counts()

In [ ]:
sizes = []
for image in os.listdir("/content/cifar-10/test"):
  image = plt.imread("/content/cifar-10/test/"+image)
  sizes.append(image.shape)
pandas.Series(sizes).value_counts()

In [ ]:
# Downloading cifar10 data 

In [ ]:
from torchvision.transforms.transforms import RandomCrop
transform = transforms.Compose(
    [#transforms.Grayscale(),
     transforms.ToTensor(),
     ])
transform_test = transforms.Compose(
    [transforms.ToTensor()
    ])

In [ ]:
# code to get cifar10 dataset:

x_train  = datasets.CIFAR10(root='./data/',train=True,download=True,transform=transform)
x_test = datasets.CIFAR10(root='./data/',train=False,download=True,transform=transform_test)
clear_output() 

In [ ]:
conv_vgg19 = nn.Sequential(
    
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

)

In [ ]:
class Custom_Model(nn.Module):
      def __init__(self):
        super(Custom_Model, self).__init__() 

        self.convlayer = nn.Sequential(
            nn.Conv2d(in_channels=3, out_channels=32, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.ReLU(),
            nn.BatchNorm2d(32),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),
            nn.Conv2d(in_channels=32, out_channels=64, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.ReLU(),
            nn.BatchNorm2d(64),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

        )

        #self.conv_vgg_19 = conv_vgg19

        self.dense_layer = nn.Sequential(
            nn.Flatten(),
            nn.Linear(4*224*224, 512),
            nn.ReLU(),
            nn.Linear(512,128),                   # Dense layer
            #nn.ReLU(),
            #nn.Linear(128,64),
            nn.ReLU(),
            nn.Linear(128,10),
            nn.Softmax(dim=1)
        )

      def forward(self, x): # x is the input image
        #y = self.conv_vgg_19(x)
        #print(x.shape)
        y = self.convlayer(x)
        y = self.dense_layer(x)
        return y

In [ ]:
class Trainer():
    def __init__(self, data):

        self.device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.trainloader, self.validloader, self.testloader = self.get_iterator(data)
        
        self.model = self.get_model().to(self.device)
        self.criterion = self.get_criterion().to(self.device)
        self.optimizer = self.get_optimizer()

        self.accuracy_valid = torchmetrics.Accuracy().to(self.device)
        self.accuracy_train = torchmetrics.Accuracy().to(self.device)
        self.accuracy_test = torchmetrics.Accuracy().to(self.device)

        self.train_loss = []
        self.train_metrics = []
        self.valid_loss = []
        self.valid_metrics = []

        self.epochs = 2

    def get_iterator(self, data):
        train, valid, test = data
        trainloader = torch.utils.data.DataLoader(train, batch_size=500, shuffle=True, drop_last=True)      # Choosen batch_size - chosen 10
        validloader = torch.utils.data.DataLoader(valid, batch_size=500, shuffle=False, drop_last=True)     # Choosen batch_size - chosen 10
        testloader = torch.utils.data.DataLoader(test, batch_size=100, shuffle=False)                       # Choosen batch_size - chosen 10
        return trainloader, validloader, testloader

    def get_criterion(self):
        return nn.CrossEntropyLoss()                                                                         # Added loss-(CrossEntropyLoss)
    
    def get_optimizer(self):
        return torch.optim.Adam(self.model.parameters(), lr=0.01)                                                       # Added optimizer(Adam)

    def get_model(self):
        model = Custom_Model()                                                                                # Added model
        return model

    def save(self, epoch):
        torch.save({
            'model_state_dict': self.model.state_dict(),
            'optimizer_state_dict': self.optimizer.state_dict(),
            }, os.path.join(PATH, "model.pth"))
        
    def load(self):
        if os.path.exists(os.path.join(PATH, "model.pth")):
            checkpoints = torch.load(os.path.join(self.args.checkpoint, "model.pth"), map_location=self.device)
            self.model.load_state_dict(checkpoints['model_state_dict'])
            self.optimizer.load_state_dict(checkpoints['optimizer_state_dict'])

    def train(self):
        epoch_loss = 0
        epoch_metrics = {}

        self.model.train()

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.trainloader), bar_char='█')
            for index, (image, label) in enumerate(self.trainloader):
                image = image.to(self.device)
                label = label.to(self.device)
                #print(image.shape)
                #image = image.reshape(image.shape[0],-1)
                self.optimizer.zero_grad()
                
                output = self.model(image) # Forward Pass

                loss = self.criterion(output,label) # Evaluating Loss
      
                loss.backward() # Back Propogation

                _,predcitions = output.max(1)
                self.accuracy_train(predcitions,label)

                self.optimizer.step()
                bar.update()

        accuracy = self.accuracy_train.compute()
        self.accuracy_train.reset()
        return epoch_loss, accuracy

    def evaluate(self):
        epoch_loss = 0
        epoch_metrics = {}        

        with torch.autograd.set_detect_anomaly(True):
           
            bar = pyprind.ProgBar(len(self.validloader), bar_char='█')
            for index, (image, label) in enumerate(self.validloader):
                image = image.to(self.device)
                label = label.to(self.device)
                #image = image.reshape(image.shape[0],-1)

                output =self.model(image).to(self.device)  

                loss = self.criterion(output,label) 

                _,predcitions = output.max(1)
                self.accuracy_valid(predcitions,label)

                bar.update()
                epoch_loss = loss

            accuracy = self.accuracy_valid.compute()
            self.accuracy_valid.reset()
            return epoch_loss, accuracy

    def test(self):

        self.model.eval()

        outputs = numpy.array([])

        with torch.autograd.set_detect_anomaly(True):
            bar = pyprind.ProgBar(len(self.testloader), bar_char='█')
            for index, (image, label) in enumerate(self.testloader):
                image = image.to(self.device)
                label = label.to(self.device)
                #image = image.reshape(image.shape[0],-1)

                output = self.model(image)   

                _,pred = output.max(1)
               
                pred = pred.cpu().numpy()
                outputs = numpy.append(outputs,pred)
               

                #print(outputs.shape)
                bar.update()
        outputs.reshape((1,10000)) 
        return outputs
    
    def fit(self):

        for epoch in range(1, self.epochs+1, 1):

            epoch_train_loss, epoch_train_metrics = self.train()

            self.train_loss.append(epoch_train_loss)
            self.train_metrics.append(epoch_train_metrics)

            epoch_valid_loss, epoch_valid_metrics = self.evaluate()
            
            self.valid_loss.append(epoch_valid_loss)
            self.valid_metrics.append(epoch_valid_metrics) 

            print(f'Epoch {epoch}/{self.epochs+1}: Train Loss = {epoch_train_loss} | Validation Loss = {epoch_valid_loss}')
            print(f'train accuracy : {epoch_train_metrics}   valid accuracy : {epoch_valid_metrics} ')

            if epoch_valid_metrics >= max(self.valid_metrics):
                self.save(epoch)

In [ ]:
train_data = CreateDataset("/content/cifar-10")                 
train_data, valid_data = torch.utils.data.random_split(train_data, [len(train_data)-len(train_data)//10, len(train_data)//10])
test_data = CreateDataset("/content/cifar-10",'test')             
data = (train_data, valid_data, test_data)

trainer = Trainer(data)
trainer.fit()

outputs = trainer.test()
print("\n",outputs)


0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:26
0% [██████████] 100% | ETA: 00:00:00

Epoch 1/3: Train Loss = 0 | Validation Loss = 2.3025999069213867
train accuracy : 0.10048888623714447   valid accuracy : 0.10019999742507935 



Total time elapsed: 00:00:02
0% [██████████████████████████████] 100% | ETA: 00:00:00
Total time elapsed: 00:00:25
0% [██████████] 100% | ETA: 00:00:00

Epoch 2/3: Train Loss = 0 | Validation Loss = 2.302568197250366
train accuracy : 0.09857777506113052   valid accuracy : 0.09780000150203705 



Total time elapsed: 00:00:02
0% [██████████████████████████████] 100% | ETA: 00:00:00


 [5. 5. 5. ... 5. 5. 5.]



Total time elapsed: 00:00:06


In [ ]:
'''VGG19 Arch

Conv3x3 (64)
Conv3x3 (64)
MaxPool
Conv3x3 (128)
Conv3x3 (128)
MaxPool
Conv3x3 (256)
Conv3x3 (256)
Conv3x3 (256)
Conv3x3 (256)
MaxPool
Conv3x3 (512)
Conv3x3 (512)
Conv3x3 (512)
Conv3x3 (512)
MaxPool
Conv3x3 (512)
Conv3x3 (512)
Conv3x3 (512)
Conv3x3 (512)
MaxPool
Fully Connected (4096)    
Fully Connected (4096)    Dense layer write acc to your model
Fully Connected (1000)
SoftMax
Resize the images to (224,224,3)
'''

In [ ]:
conv_vgg19 = nn.Sequential(
    
            nn.Conv2d(in_channels=3, out_channels=64, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=64, out_channels=64, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=64, out_channels=128, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=128, out_channels=128, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=128, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=256, out_channels=256, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.Conv2d(in_channels=512, out_channels=512, kernel_size=(3,3), stride = (1,1) , padding = (1,1) ),
            nn.MaxPool2d(kernel_size=(2,2),stride=(2,2) ),

)

In [ ]:
# output = floor[(input + 2*padding — kernel) / stride + 1]
dev = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# a class to build the basic blocks
class BlockBuilder(nn.Module):
    def __init__(self,inp_cha,out_cha,stride=1,kernel=3):
      super().__init__()

      self.stride = stride
      self.kernel = kernel
      self.inp = inp_cha
      self.out = out_cha

      # here I have defined the alternate path as path2

      if self.stride != 1 or self.inp != self.out:
        self.path2 = nn.Sequential( nn.Conv2d(self.inp, self.out, kernel_size=1, stride=self.stride),
                                   nn.BatchNorm2d(self.out).to(dev)
        )
      else:
        self.path2 = nn.Sequential().to(dev)

      # here I am going to define the conv layers for the actual path

      self.actual_layers = nn.Sequential(
          nn.Conv2d(self.inp, self.out//4, kernel_size=1, stride=1).to(dev),
          nn.BatchNorm2d(self.out//4),
          nn.ReLU(),
          nn.Conv2d( self.out//4, self.out//4, kernel_size=3, stride=self.stride,padding=1).to(dev),
          nn.BatchNorm2d(self.out//4),
          nn.ReLU(),
          nn.Conv2d( self.out//4, self.out, kernel_size=1, stride=1).to(dev),
          nn.BatchNorm2d(self.out),
          nn.ReLU(),
      ).to(dev)

    def forward(self,input):

        input_path2 = self.path2(input).to(dev)
        output = self.actual_layers(input).to(dev)
        combined_output = (output + input_path2).to(dev)
        return nn.ReLU()(combined_output).to(dev)

# Block Builder finished

In [ ]:
# here I am going to buid the whole network

class network(nn.Module):
    def __init__(self,block=BlockBuilder,arch=[64, 256, 512, 1024, 2048],repeaters=[3,4,6,3],input_channels=3,strides=[1,2,1,2],classes=10):
        super(network,self).__init__()

        self.inpc = input_channels
        self.arch = arch
        self.repeaters = repeaters
        self.strides = strides
        self.cl = classes

        # here I have defined the first layer0

        self.layero = nn.Sequential(
            nn.Conv2d(self.inpc, 64, kernel_size=7, stride=2, padding=3),
            nn.MaxPool2d(kernel_size=3, stride=2, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU()
            )
        self.dynamic_layers = {}
        # here I am defining the blocks
        self.dlayers = nn.Sequential()

        for i in range(len(self.arch)-1):
          self.dlayers.add_module(f'conv{i}', block(self.arch[i],self.arch[i+1], stride=self.strides[i]))
          for j in range(0,self.repeaters[i]-1):
              self.dlayers.add_module(f'conv{i}+{j}', block(self.arch[i+1],self.arch[i+1], stride=1))
          self.dlayers.to(dev)

            # definfig the Fc layer here
          self.avgpool = torch.nn.AdaptiveAvgPool2d(1)

          self.fc = nn.Sequential(
              nn.Flatten(),
              nn.Linear(self.arch[4],4096),
              nn.ReLU(),
              nn.Linear(4096,512),
              nn.ReLU(),
              nn.Linear(512,64),
              nn.ReLU(),
              nn.Linear(64,10),
              nn.Softmax(dim=1),
          )

    def forward(self,input_s):
        input_s = self.layero(input_s)
        input_s = self.dlayers(input_s)
        input_s = self.avgpool(input_s)
        input_s = self.fc(input_s)
        return input_s
    

In [ ]:
restnet_model = network().to(torch.device("cuda:0" if torch.cuda.is_available() else "cpu"))

In [ ]:
restnet_model

network(
  (layero): Sequential(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3))
    (1): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (3): ReLU()
  )
  (dlayers): ModuleList(
    (conv0): BlockBuilder(
      (path2): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      )
      (actual_layers): Sequential(
        (0): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1))
        (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (2): ReLU()
        (3): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
        (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (5): ReLU()
        (6): Conv2d(64, 256, kernel_size=(1, 1), str